# <a id='toc1_'></a>[Diablo](#toc0_)


**Table of contents**<a id='toc0_'></a>    
- [Diablo](#toc1_)    
  - [Introdution](#toc1_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Introdution](#toc0_)

Diablo est un jeu vidéo de type hack 'n' slash développé par Blizzard North. La version PC fonctionnant sous Windows a été publiée en janvier 1997 en Amérique du Nord et en Europe par Blizzard Entertainment. Une version pour PlayStation, développée par Electronic Arts, et une version Macintosh sont publiées l'année suivante en mars 1998.

Diablo se déroule dans un univers médiéval-fantastique où le joueur explore des donjons peuplés de créatures qu'il doit combattre pour gagner de l'expérience et des trésors, l'objectif du jeu étant de retrouver Diablo et le tuer. Le jeu est considéré comme ayant réinventé le genre du jeu vidéo de rôle en y introduisant le principe du point and click et en le rendant plus accessible. Diablo a ainsi fortement influencé l'industrie du jeu vidéo, son système de jeu ayant été repris dans de nombreux titres qualifiés de « clones » ou de « Diablo-like ».

Le jeu est extrêmement bien accueilli par les critiques et connaît dès sa sortie un solide succès commercial. En 1997, il décroche ainsi la troisième place des meilleures ventes de jeux vidéo sur PC ; le 29 août 2001, il s'est vendu à plus de 2,5 millions d'exemplaires dans le monde. Par la suite, Blizzard North lance le développement de Diablo II qui sort le 29 juin 2000. Le troisième épisode de la série, Diablo III, annoncé le 28 juin 2008, est sorti le 15 mai 2012. Le quatrième épisode de la série, Diablo IV, est annoncé en novembre 2019 pour une sortie en juin 2023.

La carte du jeu est implémentée sous la forme de tuiles, il faut le voir comme un grand graphe avec des sommets (tuiles) qui peuvent être accessibles ou pas (avec des obstacles). Il est donc logique de penser à une implémentation d'un algorithme mathématique pour rechercher le plus court chemin d'une tuile à une autre.
Diablo I est aujourd'hui considéré comme un des premiers jeux vidéos utilisant l'algorithme A* pour trouver les chemins les plus courts. 

### Explication du code source

Voici la fonction de recherche de chemin qui est utilisée dans le code pour plusieurs choses, notamment le calcul du chemin lorsque l'on clique sur une tuile du jeu, mais aussi lorsque les monstres poursuivent le joueur.

```c++
int FindPath(BOOL (*PosOk)(int, int, int), int PosOkArg, int sx, int sy, int dx, int dy, char *path)
{
	PATHNODE *path_start, *next_node, *current;
	int path_length, i;

	// clear all nodes, create root nodes for the visited/frontier linked lists
	gdwCurNodes = 0;
	path_2_nodes = path_new_step();
	pnode_ptr = path_new_step();
	gdwCurPathStep = 0;
	path_start = path_new_step();
	path_start->g = 0;
	path_start->h = path_get_h_cost(sx, sy, dx, dy);
	path_start->x = sx;
	path_start->f = path_start->h + path_start->g;
	path_start->y = sy;
	path_2_nodes->NextNode = path_start;
	// A* search until we find (dx,dy) or fail
	while ((next_node = GetNextPath())) {
		// reached the end, success!
		if (next_node->x == dx && next_node->y == dy) {
			current = next_node;
			path_length = 0;
			while (current->Parent) {
				if (path_length >= MAX_PATH_LENGTH)
					break;
				pnode_vals[path_length++] = path_directions[3 * (current->y - current->Parent->y) - current->Parent->x + 4 + current->x];
				current = current->Parent;
			}
			if (path_length != MAX_PATH_LENGTH) {
				for (i = 0; i < path_length; i++)
					path[i] = pnode_vals[path_length - i - 1];
				return i;
			}
			return 0;
		}
		// ran out of nodes, abort!
		if (!path_get_path(PosOk, PosOkArg, next_node, dx, dy))
			return 0;
	}
	// frontier is empty, no path!
	return 0;
}
```

#### Les paramètres de la fonction

##### *PosOk
##### PosOkArg
##### sx
##### sy
##### dx
##### dy
##### *path


#### Les variables de la fonction

##### gwdCurNodes
##### path_2_nodes
##### pnode_ptr
##### gdwCurPathStep
##### path_start
##### next_node
##### current
##### path_length
##### pnode_vals


#### Les conditions

##### next_node = GetNextPath()
##### path_length >= MAX_PATH_LENGTH
##### path_length != MAX_PATH_LENGTH
Si le longueur du chemin depasse une longueur maximale, on considère que la tuile cible est trop loin, cela permet de rendre plus réaliste la poursuite des ennemis, en permettant au joueur de fuir loin pour ne plus se faire poursuivre.
##### !path_get_path(PosOk, PosOkArg, next_node, dx, dy)

#### La boucle

##### current->Parent

#### Le retour

##### i
##### 0


si chemin trop long : return 0 donc rien ne se passe

### Cas d'utilisation

- Monstres qui poursuivent le joueur 
- Déplacement du personnage sur la grille (carte) quand on clique sur une case


il est utilisé lorsque que l'on clique sur une case de la map, le jeu va aemener notre personnage a la case cliquée par le chemin le plus court possible    

path.h : interface des algos
path.cpp : implémentation des algos
player.cpp : fonction FixPlayerLocation
